In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('drive/MyDrive/Graph EdgeList/Positive_Negative_Interactions_11000.csv')
data = data.iloc[: , 1:]

In [3]:
data.head(-5)

,Drug ID,Target ID,Label
0,DB00001,P00734,1
1,DB00002,P00533,1
2,DB00002,O75015,1
3,DB00002,P02745,1
4,DB00002,P02746,1
...,...,...,...
22048,DB01621,P24024,-1
22049,DB00334,P80192,-1
22050,DB00600,P07332,-1
22051,DB00794,Q9P289,-1


In [15]:
column = ['F'+str(i) for i in range(1,65)]

column.insert(0,'node_id')

In [ ]:
column

In [16]:
embedding=pd.read_csv('/content/drive/MyDrive/Graph EdgeList/nodes_embedding_64.csv',index_col=False,names=column,dtype='a')


In [17]:
embedding.head(-5)

,node_id,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,F11,F12,F13,F14,F15,F16,F17,F18,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28,F29,F30,F31,F32,F33,F34,F35,F36,F37,F38,F39,F40,F41,F42,F43,F44,F45,F46,F47,F48,F49,F50,F51,F52,F53,F54,F55,F56,F57,F58,F59,F60,F61,F62,F63,F64
0,DB00001,-0.540512085,0.092157982,0.207670316,0.862200201,0.027476951,0.264243633,-0.318120003,0.782378972,0.528774619,-0.555405259,-0.564396441,-0.962679148,0.436288506,0.458396554,-0.726142466,0.210273206,0.508894861,-0.257993758,-0.15556258,0.070200302,-0.559980392,-0.055085674,0.501433134,-0.610793591,0.20014666,-0.130834371,-0.459934145,0.21917133,-0.446948349,0.844013035,0.498319417,0.160800755,0.28933692,1.255531669,-0.376350343,0.037448294,-0.727737248,0.304960549,0.580407917,0.018787345,0.275166512,0.224416226,0.448559046,0.224880844,0.850475967,-0.099512734,-0.124339484,-0.045414194,-0.656435549,0.464744151,0.23419401,-0.327765107,-0.386757255,0.626896322,1.332466364,1.002473235,-0.096364915,-0.273392975,-0.38266024,0.259769142,-0.574071705,-0.217167005,0.235089049,-0.425414085
1,P00734,-0.677274287,0.286098182,0.491006643,0.88932842,-0.065397844,0.150812671,-0.167208388,0.490411133,0.063002966,-0.333331972,-0.46278882,-1.024350166,0.528920889,0.857328713,-0.593675494,0.408331722,0.458371788,-0.357535928,-0.056777753,0.170099974,-0.397774071,0.413993448,0.406806648,-0.520999551,-0.011135323,-0.361132205,0.019842273,0.64562279,-0.258972675,0.42458725,0.131408274,0.208086446,0.306702107,0.604653955,0.054796293,0.400164217,-0.300553858,0.274479568,0.561712205,0.310659617,-0.163870454,0.536101997,0.193589419,0.172701478,0.697003543,-0.150853023,-0.336513579,0.001438345,-0.805161595,-0.090662129,0.199891672,-0.629672468,0.054102391,0.800537527,0.832549214,0.343210191,0.078096338,-0.036137812,-0.070790306,-0.013410497,-0.627400339,-0.364664316,0.01688602,-0.28516233
2,DB00002,0.083409958,0.493797541,0.228755355,-0.018126983,-0.48317495,0.16607672,0.235328108,1.069189191,-0.201625228,1.030033946,-0.594906032,0.946046531,1.185775757,0.574977517,-0.07192985,-0.453185618,0.489487082,-0.785886228,0.163006783,0.268815905,0.225327179,0.893892884,-0.29832691,-0.140379325,-0.086856529,-0.05338379,0.149545342,-0.810371935,-0.754116476,-1.030129433,0.750492513,0.853495896,-0.547229946,0.17062968,-0.194016695,0.025348317,0.75567323,0.617618322,0.048688795,-0.238782763,-0.216283202,0.037469454,0.611126781,0.327397645,-0.308894992,-0.165362075,-1.162209034,-0.058319762,0.006694297,0.710047364,-0.192551062,0.679711998,0.213794351,0.083879471,0.703756154,0.705516636,0.287372708,-0.554012656,0.048669655,-0.026507666,0.243556887,0.011453116,-0.531214476,0.477717489
3,P00533,0.564742327,-0.102836065,-0.006208587,0.391906142,-0.673503339,0.193608612,-0.161899462,0.226795852,-1.257881522,0.453504384,-0.093286857,1.251747966,-0.443451852,0.22318171,-0.502012193,-0.420516849,0.201756939,0.525140941,-0.634975791,-0.217431501,-0.041843407,-0.02000672,-0.486172438,-0.573430717,-0.555259466,0.298822373,0.396895856,0.16573067,0.320142925,-0.095579863,0.201655716,0.447290808,-0.69285816,0.114005424,0.144118592,0.443403542,0.575106025,-0.160592511,0.416121274,0.416795671,0.747294068,0.254563779,0.68612045,-0.264816374,-0.226638138,-0.2843104,-1.052616835,-0.333113015,0.123390153,0.49978441,0.433900714,0.828002095,0.129417524,0.054715205,-0.162010327,0.083524078,0.232129559,-0.426136255,0.57806313,-0.323532134,0.063349456,-0.30723089,-0.071109779,0.379133612
4,O75015,0.274759918,0.589039207,0.282539427,-0.334546894,-0.833500683,0.174489543,-0.532150328,1.200007558,0.628423393,0.485096484,-0.437084436,0.142634645,0.755268157,0.677057028,-0.262400448,-0.136697277,0.577059865,-0.715508938,0.464510947,0.105569333,0.468254209,0.326742262,0.110923916,-0.402740359,0.280113935,-0.214937836,-0.093154497,-0.462000668,-0.565593481,-1.794117689,0.579479098,1.549741983,-0.620021641,0.076847129,0.027174443,-0.026432773,0.569025636,0.69025439,0.167480662,-0.334337264,-0.20314613,0.150622964,0.482416064,0.519087076,-

In [18]:
for i in range(1,65):
    embedding['F'+str(i)] = embedding['F'+str(i)].apply(lambda x : float(x))
    
    
#embedding['node_id'] = embedding['node_id'].apply(lambda x : float(x))

cols = ['F'+str(i) for i in range(1,65)]    
cols.insert(0,'Drug ID')
cols.insert(1,'Target ID')
cols.insert(2,'Label')
dataset_add =  pd.DataFrame(columns=cols,data=None) 

In [19]:
dataset_add

,Drug ID,Target ID,Label,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,F11,F12,F13,F14,F15,F16,F17,F18,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28,F29,F30,F31,F32,F33,F34,F35,F36,F37,F38,F39,F40,F41,F42,F43,F44,F45,F46,F47,F48,F49,F50,F51,F52,F53,F54,F55,F56,F57,F58,F59,F60,F61,F62,F63,F64


In [20]:
from tqdm import tqdm
for i,row in tqdm(data.iterrows()):
    label=row['Label']
    target_vector = embedding.loc[embedding['node_id'] == row['Target ID']]
    target_vector = target_vector.reset_index(drop=True)
    drug_vector = embedding.loc[embedding['node_id'] == row['Drug ID']]
    drug_vector = drug_vector.reset_index(drop=True)
    
    dataset_vector = drug_vector.add(target_vector)
    
    dataset_vector= dataset_vector.drop(columns = ['node_id'])
    dataset_vector['Label'] = label
    dataset_vector['Drug ID'] = drug_vector['node_id']
    dataset_vector['Target ID'] = target_vector['node_id']
    
    
    dataset_add= dataset_add.append(dataset_vector,ignore_index=True)

    

22058it [03:49, 96.08it/s]


In [21]:
dataset_add.head(-5)

,Drug ID,Target ID,Label,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,F11,F12,F13,F14,F15,F16,F17,F18,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28,F29,F30,F31,F32,F33,F34,F35,F36,F37,F38,F39,F40,F41,F42,F43,F44,F45,F46,F47,F48,F49,F50,F51,F52,F53,F54,F55,F56,F57,F58,F59,F60,F61,F62,F63,F64
0,DB00001,P00734,1,-1.217786,0.378256,0.698677,1.751529,-0.037921,0.415056,-0.485328,1.272790,0.591778,-0.888737,-1.027185,-1.987029,0.965209,1.315725,-1.319818,0.618605,0.967267,-0.615530,-0.212340,0.240300,-0.957754,0.358908,0.908240,-1.131793,0.189011,-0.491967,-0.440092,0.864794,-0.705921,1.268600,0.629728,0.368887,0.596039,1.860186,-0.321554,0.437613,-1.028291,0.579440,1.142120,0.329447,0.111296,0.760518,0.642148,0.397582,1.547480,-0.250366,-0.460853,-0.043976,-1.461597,0.374082,0.434086,-0.957438,-0.332655,1.427434,2.165016,1.345683,-0.018269,-0.309531,-0.453451,0.246359,-1.201472,-0.581831,0.251975,-0.710576
1,DB00002,P00533,1,0.648152,0.390961,0.222547,0.373779,-1.156678,0.359685,0.073429,1.295985,-1.459507,1.483538,-0.688193,2.197794,0.742324,0.798159,-0.573942,-0.873702,0.691244,-0.260745,-0.471969,0.051384,0.183484,0.873886,-0.784499,-0.713810,-0.642116,0.245439,0.546441,-0.644641,-0.433974,-1.125709,0.952148,1.300787,-1.240088,0.284635,-0.049898,0.468752,1.330779,0.457026,0.464810,0.178013,0.531011,0.292033,1.297247,0.062581,-0.535533,-0.449672,-2.214826,-0.391433,0.130084,1.209832,0.241350,1.507714,0.343212,0.138595,0.541746,0.789041,0.519502,-0.980149,0.626733,-0.350040,0.306906,-0.295778,-0.602324,0.856851
2,DB00002,O75015,1,0.358170,1.082837,0.511295,-0.352674,-1.316676,0.340566,-0.296822,2.269197,0.426798,1.515130,-1.031990,1.088681,1.941044,1.252035,-0.334330,-0.589883,1.066547,-1.501395,0.627518,0.374385,0.693581,1.220635,-0.187403,-0.543120,0.193257,-0.268322,0.056391,-1.272373,-1.319710,-2.824247,1.329972,2.403238,-1.167252,0.247477,-0.166842,-0.001084,1.324699,1.307873,0.216169,-0.573120,-0.419429,0.188092,1.093543,0.846485,-1.245692,-0.466440,-1.872275,0.108079,-0.033260,1.232308,-0.267434,0.813787,0.266467,0.256560,1.605645,0.786444,1.001873,-0.930362,-0.075718,0.444748,-0.365249,0.248512,-0.285290,0.877249
3,DB00002,P02745,1,0.293993,1.047876,0.473940,-0.961737,-1.112729,0.094836,0.011162,2.130513,-0.026086,1.914564,-1.080309,1.669392,2.333219,1.051460,0.042632,-0.630490,1.407062,-1.416507,0.939928,0.646612,0.853872,1.861553,-0.622253,-0.543009,-0.016107,-0.284430,0.224380,-1.679317,-1.771496,-1.750010,1.150163,1.982100,-0.450728,0.533144,-0.527731,0.024697,1.590564,1.293855,-0.394317,-0.276451,-0.713536,0.795887,1.316806,0.539630,-0.351549,-0.426680,-1.891991,-0.633446,0.151016,1.026000,-0.699747,0.984124,0.841034,-0.105650,1.765294,1.127378,0.651434,-0.197828,0.129433,0.477084,0.184170,0.098055,-0.364112,0.630863
4,DB00002,P02746,1,0.095389,0.923703,0.418403,-0.299447,-0.490171,0.464269,0.482566,1.852560,-0.189535,1.932169,-0.971075,1.177750,1.947225,0.680946,0.134628,-0.954644,0.899323,-1.594825,0.339024,0.072039,1.127319,1.866261,-0.502087,-0.280141,-0.129898,-0.662575,0.578693,-1.101126,-2.131579,-1.594313,1.079803,1.247589,-0.603913,0.436105,-0.030480,-0.186765,0.879212,1.079411,0.012803,-0.200793,0.067877,0.445334,1.119573,0.657920,-0.232548,0.012467,-1.920812,-0.666098,0.014081,0.850271,-0.170800,0.863038,0.584641,0.304442,1.562473,1.306523,0.983234,-0.770081,0.034218,0.405968,0.173748,0.093227,-0.432726,0.540630
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22048,DB01621,P24024,-1,-0.850574,-1.850052,-0.969218,0.115276,0.090584,-0.432322,-0.309347,0.208837,0.227392,0.233711,-0.265583,1.037599,-0.645470,-0.000989,0.042124,-0.325794,0.872067,-0.918577,-0.994268,1.171711,0.774600,0.254273,1.052169,-0.076236,0.648392,0.375111,0.388447,0.620596,1.149126,0.559644,0.364530,-0.793265,0.169673,0.580175,0.573576,-0

In [22]:
embedding.loc[0]+embedding.loc[1]

node_id    DB00001P00734
F1              -1.21779
F2              0.378256
F3              0.698677
F4               1.75153
               ...      
F60             0.246359
F61             -1.20147
F62            -0.581831
F63             0.251975
F64            -0.710576
Length: 65, dtype: object

In [23]:
dataset_add.to_csv('drive/MyDrive/Graph EdgeList/Embedding_DT_label_11000_64Feature.csv',index=False)  